In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 1. Create normalization function

In [15]:
import pandas as pd

def normalize_landmarks(landmarks, anchor_point):
    normalized = []
    for lm in landmarks:
        # Normalize each landmark by subtracting anchor_point's coordinates
        normalized.append((lm.x - anchor_point.x, lm.y - anchor_point.y, lm.z - anchor_point.z))
    return normalized


# 2. Track coordinates on landmarks (through webcam pose tracking)

In [20]:
# Define global variable to store normalized landmarks
normalized_landmarks = None

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
cap = cv.VideoCapture(0)


# Define your anchor point (set to Nose)
anchor_point = None  # We'll set this in the first frame

# Start video capture and pose processing
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Convert image color for MediaPipe processing
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process the image
        results = pose.process(image)

        # If landmarks are found, normalize them and store in the global variable
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Set the anchor point to nose in the first frame
            if anchor_point is None:
                anchor_point = landmarks[mp_pose.PoseLandmark.NOSE]
            
            # Normalize the landmarks and store them in the global variable
            global normalized_landmarks
            anchor_point = landmarks[mp_pose.PoseLandmark.NOSE]
            normalized_landmarks = normalize_landmarks(landmarks, anchor_point)

            # Draw landmarks on the image
            image.flags.writeable = True
            image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Show the processed image
        cv.imshow('Pose Tracking', image)
        
        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()

# 3. Convert coords to dataframe

In [22]:
coords_df = pd.DataFrame(normalized_landmarks, columns=['x', 'y', 'z'])
coords_df

,x,y,z
0,0.000000,0.000000,0.000000
1,0.023418,-0.029196,0.017282
2,0.035589,-0.027091,0.016894
3,0.046533,-0.024130,0.016528
4,-0.016349,-0.032521,0.038632
5,-0.029148,-0.031873,0.038624
6,-0.039795,-0.030660,0.038257
7,0.065384,0.001891,0.139228
8,-0.050841,-0.007621,0.243784
9,0.021132,0.051367,0.047212
